## 03_builder.ipynb

This notebook demonstrates how to use the **Builder** module in the `pysubmit` package.

The builder pattern provides a clean, modular way to inject logic into the simulation setup process.  
It gives users "hooks" to modify or interact with the AEDT design in a structured and reusable way.

In this tutorial, we cover three different builder classes:

1. `DesignVariableBuilder`: Modify design parameters
2. `FunctionBuilder`: Run custom Python functions on the design
3. `ScriptBuilder`: Load and run user-defined Python functions stored in external `.py` files

> **Note:**  
> These builders are especially useful when writing parameter sweeps or automating simulation workflows — topics we'll explore in later tutorials.


In [1]:
from ansys.aedt.core.hfss import Hfss

hfss_file_path = 'sime_design_readout_for_builder.aedt'

### 1. Design Variable Builder

This builder is used to define and apply design variables in your AEDT simulation.

You can specify parameters as a dictionary, and the builder will apply them to the design automatically.


In [2]:
from ansys.aedt.core.hfss import Hfss
from pysubmit.workflow.builder import DesignVariableBuilder


builder = DesignVariableBuilder(
    setup_name='Setup1',
    design_name='my_design'
)


We define a dictionary of parameters to apply to the design.


In [3]:
parameters = {'chip_base_length': '20mm'}

with Hfss(
    project=hfss_file_path,
    design='my_design',
    remove_lock=True,
    close_on_exit=True
) as hfss:
    
    # viewing the before value
    print(f'\n\n{"#" * 30}')
    print(f'CHIP BASE VALUE BEFORE: {hfss["chip_base_length"]}')
    print(f'{"#" * 30}\n\n')

    builded_parameters = builder.build(hfss, parameters=parameters)

    # viewing the after value
    print(f'\n\n{"#" * 30}')
    print(f'CHIP BASE VALUE AFTER: {hfss["chip_base_length"]}')
    print(f'{"#" * 30}\n\n')

PyAEDT INFO: Parsing sime_design_readout_for_builder.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.1.
PyAEDT INFO: File sime_design_readout_for_builder.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_3530d2b5-8155-48df-93a8-b7be7bfdb879.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 61840.
PyAEDT INFO: Electronics Desktop started on gRPC port: 61840 after 10.015042781829834 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 25608.
PyAEDT IN

### 2. Function Builder

The `FunctionBuilder` lets you define a custom function that receives the design object  
and performs arbitrary actions on it — ideal for scripting your own automation logic.


In [4]:
from pysubmit.workflow.builder import FunctionBuilder

def my_custom_function(hfss, value):
    hfss['chip_base_length'] = value
    return {'chip_base_length': value}



function_builder = FunctionBuilder(
    function=my_custom_function
)

We now use the function builder to execute the custom function within the HFSS design context.


In [5]:
with Hfss(
    project=hfss_file_path,
    design='my_design',
    remove_lock=True,
    close_on_exit=True
) as hfss:
    
    # viewing the before value
    print(f'\n\n{"#" * 30}')
    print(f'CHIP BASE VALUE BEFORE: {hfss["chip_base_length"]}')
    print(f'{"#" * 30}\n\n')

    builded_parameters = function_builder.build(hfss, parameters={'value': '20mm'})

    # viewing the after value
    print(f'\n\n{"#" * 30}')
    print(f'CHIP BASE VALUE AFTER: {hfss["chip_base_length"]}')
    print(f'{"#" * 30}\n\n')


PyAEDT INFO: Parsing sime_design_readout_for_builder.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.1.
PyAEDT INFO: File sime_design_readout_for_builder.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_3530d2b5-8155-48df-93a8-b7be7bfdb879.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 61907.
PyAEDT INFO: Electronics Desktop started on gRPC port: 61907 after 9.632612705230713 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 7100.
PyAEDT INFO

### 3. Script Builder

The `ScriptBuilder` is used when your simulation logic is stored in a separate Python file.

Instead of importing your function manually into the notebook, you can simply provide the file path.  
`ScriptBuilder` will import and execute the function from that file in the context of the current HFSS design.

This is particularly helpful for modular projects where simulation setup logic is kept in external scripts.


In [6]:
from pysubmit.workflow.builder.script_builder import ScriptBuilder


script_builder = ScriptBuilder(
    path='script_builder_function.py'
)


with Hfss(
    project=hfss_file_path,
    design='my_design',
    remove_lock=True,
    close_on_exit=True
) as hfss:
    
    # viewing the before value
    print(f'\n\n{"#" * 30}')
    print(f'CHIP BASE VALUE BEFORE: {hfss["chip_base_length"]}')
    print(f'{"#" * 30}\n\n')

    builded_parameters = script_builder.build(hfss, parameters={'value': '20mm'})

    # viewing the after value
    print(f'\n\n{"#" * 30}')
    print(f'CHIP BASE VALUE AFTER: {hfss["chip_base_length"]}')
    print(f'{"#" * 30}\n\n')

PyAEDT INFO: Parsing sime_design_readout_for_builder.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.1.
PyAEDT INFO: File sime_design_readout_for_builder.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_3530d2b5-8155-48df-93a8-b7be7bfdb879.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 61969.
PyAEDT INFO: Electronics Desktop started on gRPC port: 61969 after 9.710781335830688 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 636.
PyAEDT INFO: